In [ ]:
# Get Data
!pip install vaderSentiment

import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import nltk
import seaborn as sns
import matplotlib.dates as mdates
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk.stem.porter import *
from wordcloud import WordCloud

In [ ]:
data = pd.read_csv("Dataset.csv", encoding='ISO-8859-2')
data.head()

In [ ]:
# Remove pattern function
def remove_pattern(input_txt, pattern):
  r = re.findall(pattern, input_txt)
  for i in r:
    input_txt = re.sub(i, '', input_txt)
  return input_txt

In [ ]:
# Remove emoticons 
data['Message'] = data['Message'].str.replace("[^\w()'+]", " ")
data

In [ ]:
# Remove links
data['Message'] = data['Message'].str.replace("http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*,]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", " ")
data.head()

In [ ]:
# Remove special characters, numbers, punctuations (except exclamation mark)
data['Message'] = data['Message'].str.replace("[^a-zA-Z#!]", " ")
data.head()

In [ ]:
# Remove short words
data['Message'] = data['Message'].apply(lambda x: ' '.join([w for w in x.split() if len(w) > 3]))
data.head()


In [ ]:

# Tokenize Data
tokenize_meesage = data['Message'].apply(lambda x: x.split())
tokenize_meesage.head()

In [ ]:
# Stemming Data

stemmer = PorterStemmer()

tokenize_meesage = tokenize_meesage.apply(lambda x: [stemmer.stem(i) for i in x])
tokenize_meesage.head()

In [ ]:
for i in range(len(tokenize_meesage)):
  tokenize_meesage[i] = ' '.join(tokenize_meesage[i])

data['Message'] = tokenize_meesage
data.head()

In [ ]:

# WordCloud
all_words = ' '.join([text for text in data['Message']])

wordcloud = WordCloud(width=800, height=500, random_state=21, background_color='white',  max_font_size=110).generate(all_words)

plt.figure(figsize=(10,7))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()


In [ ]:
# Hashtags mentions

def hashtag_extract(x):
  hashtags = []

  for i in x:
    ht = re.findall(r"#(\w+)", i)
    hashtags.append(ht)

  return hashtags

# Extract hashtags

hashtags = hashtag_extract(data['Message'])

# Unnesting list

hashtags = sum(hashtags, [])

In [ ]:
# Calculate Polarity

analyser = SentimentIntensityAnalyzer()

scores = []
for sentence in data['Message']:
  score = analyser.polarity_scores(sentence)
  scores.append(score)

scores = pd.DataFrame(scores)

data['Compound'] = scores['compound']
data['Negative'] = scores['neg']
data['Positive'] = scores['pos']
data.head()

In [ ]:
# Calculate Mentions

def word_extract(x):
  words = []

  for i in x:
    ht = re.findall(r"(\w+)", i)
    words.append(ht)

  return words

# Extract words

words = word_extract(data['Message'])

# words

# Unnesting list

words = sum(words, [])

In [ ]:
# Frequency Distribution

a = nltk.FreqDist(words)

b = pd.DataFrame({'Word': list(a.keys()), 'Count': list(a.values())})
length = len(b)

# Top 10 Most Frequent Words
b = b.nlargest(columns="Count", n = 10)
plt.figure(figsize=(10,5))
axis = sns.barplot(data=b, x='Word', y='Count')
axis.set(ylabel='Count')
plt.title('Top 10 Most Frequent Words')
plt.show()

In [ ]:
# Set type of polarity
polarity = []

for i in range(len(data)):
  if (data['Compound'][i] < 0):
    polarity.append("0")
  elif (data['Compound'][i] > 0):
    polarity.append("1")

polarity = pd.DataFrame(polarity)
data['Sentiment'] = polarity
data.head()

In [ ]:
data.to_csv('message.csv', index= False)